In [2]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.models as models
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
print('Done!')

Done!


In [ ]:
# Load of all the .csv files containing extracted image features
data_USG = pd.read_csv("Features/Ultrasound features.csv")
data_MMG = pd.read_csv("Features/Mammogram features.csv")
data_multimodal = pd.read_csv("Features/multimodal features.csv")
print('Done!')

In [ ]:
#For one hot encoding the labels this tab is required
class_mapping = {
    'B': 0,
    'M': 1,
}
num_classes = 2

label_encoder = LabelEncoder()
data_USG['Class'] = label_encoder.fit_transform(data_USG['Class'])
data_MMG['Class'] = label_encoder.fit_transform(data_MMG['Class'])
data_multimodal['Class'] = label_encoder.fit_transform(data_multimodal['Class'])
print('Done!')

In [ ]:
class SelfAttentionBlock(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads=2):
        super(SelfAttentionBlock, self).__init__()
        self.num_heads = num_heads
        self.head_dim = input_dim // num_heads
        
        # Multihead self-attention mechanism
        self.multihead_attention = nn.MultiheadAttention(input_dim, num_heads)
        
        # Feedforward neural network
        self.feedforward = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.Linear(output_dim, input_dim)
        )
        
        # Layer normalization
        self.layer_norm1 = nn.LayerNorm(input_dim)
        self.layer_norm2 = nn.LayerNorm(input_dim)
        
    def forward(self, x):
        # Get the input shape dynamically
        batch_size, feature_dim = x.size()
        
        # Reshape the input to include a sequence length dimension
        x = x.view(batch_size, 1, feature_dim)
        
        # Apply layer normalization
        x_norm = self.layer_norm1(x)
        
        # Multihead self-attention
        attention_output, _ = self.multihead_attention(x_norm, x_norm, x_norm)
        
        # Residual connection and layer normalization
        x_residual = x + attention_output
        x_norm2 = self.layer_norm2(x_residual)
        
        # Remove the sequence length dimension
        output = x_norm2.view(batch_size, feature_dim)
        
        # Feedforward network
        output = self.feedforward(output)
        
        return output

In [ ]:
class SNAILFewShot(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SNAILFewShot, self).__init__()
        
        # Define the SelfAttentionBlock
        self.self_attention = SelfAttentionBlock(input_dim, output_dim)
        
        # Additional layers if needed for adaptation
        self.fc1 = nn.Linear(input_dim, output_dim)
        
        # Classification layer for few-shot learning
        self.fc2 = nn.Linear(output_dim, output_dim)  # Output dimension may vary
        
    def forward(self, x):
        # Apply SelfAttentionBlock
        x = self.self_attention(x)
        
        # Additional adaptation layers
        x = self.fc1(x)
        
        # Classification layer
        x = self.fc2(x)
        
        return x

In [ ]:
# 1. Preprocess the data from the DataFrame
# Extract the features (512 features) and labels from the DataFrame
features_USG = data_USG.iloc[:, :-1].values  # Exclude the last column (labels)
labels_USG = data_USG.iloc[:, -1].values  # Extract the labels

# Convert the features and labels to PyTorch tensors
input_data = torch.FloatTensor(features_USG)
ground_truth_labels = torch.LongTensor(labels_USG)  # Assuming labels are 0 and 1

# 2. Create an instance of the SNAILWithResNet model
# Define the input dimension (512 features) and output dimension (2 for binary classification)
input_dim = 512
output_dim = 2
snail_model = SNAILFewShot(input_dim, output_dim)

# Define the number of epochs
epochs = 10

# 3. Training loop for multiple epochs
for epoch in range(epochs):
    # Pass the data to the model to obtain predictions
    with torch.no_grad():
        predictions = snail_model(input_data)

    # 4. Evaluate the model's predictions for binary classification
    # Apply a threshold (e.g., 0.5) to obtain binary predictions
    threshold = 0.5
    binary_predictions = (predictions[:, 1] > threshold).to(torch.int)  # Assuming class 1 is the positive class

    # Calculate accuracy
    accuracy = accuracy_score(ground_truth_labels, binary_predictions)

    # Print the accuracy as a whole number
    print(f"Epoch {epoch + 1}/{epochs}, Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# 1. Preprocess the data from the DataFrame
# Extract the features (512 features) and labels from the DataFrame
features_MMG = data_MMG.iloc[:, :-1].values  # Exclude the last column (labels)
labels_MMG = data_MMG.iloc[:, -1].values  # Extract the labels

# Convert the features and labels to PyTorch tensors
input_data = torch.FloatTensor(features_MMG)
ground_truth_labels = torch.LongTensor(labels_MMG)  # Assuming labels are 0 and 1

# 2. Create an instance of the SNAILWithResNet model
# Define the input dimension (512 features) and output dimension (2 for binary classification)
input_dim = 512
output_dim = 2
snail_model = SNAILFewShot(input_dim, output_dim)

# Define the number of epochs
epochs = 10

# 3. Training loop for multiple epochs
for epoch in range(epochs):
    # Pass the data to the model to obtain predictions
    with torch.no_grad():
        predictions = snail_model(input_data)

    # 4. Evaluate the model's predictions for binary classification
    # Apply a threshold (e.g., 0.5) to obtain binary predictions
    threshold = 0.5
    binary_predictions = (predictions[:, 1] > threshold).to(torch.int)  # Assuming class 1 is the positive class

    # Calculate accuracy
    accuracy = accuracy_score(ground_truth_labels, binary_predictions)

    # Print the accuracy as a whole number
    print(f"Epoch {epoch + 1}/{epochs}, Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# 1. Preprocess the data from the DataFrame
# Extract the features (1024 features) and labels from the DataFrame
features = data_multimodal.iloc[:, :-1].values  # Exclude the last column (labels)
labels = data_multimodal.iloc[:, -1].values  # Extract the labels

# Convert the features and labels to PyTorch tensors
input_data = torch.FloatTensor(features)
ground_truth_labels = torch.LongTensor(labels)  # Assuming labels are 0 and 1

# 2. Create an instance of the SNAILWithResNet model
# Define the input dimension (512 features) and output dimension (2 for binary classification)
input_dim = 1024
output_dim = 2
snail_model = SNAILFewShot(input_dim, output_dim)

# Define the number of epochs
epochs = 20

# 3. Training loop for multiple epochs
for epoch in range(epochs):
    # Pass the data to the model to obtain predictions
    with torch.no_grad():
        predictions = snail_model(input_data)

    # 4. Evaluate the model's predictions for binary classification
    # Apply a threshold (e.g., 0.5) to obtain binary predictions
    threshold = 0.5
    binary_predictions = (predictions[:, 1] > threshold).to(torch.int)  # Assuming class 1 is the positive class

    # Calculate accuracy
    accuracy = accuracy_score(ground_truth_labels, binary_predictions)

    # Print the accuracy as a whole number
    print(f"Epoch {epoch + 1}/{epochs}, Accuracy: {accuracy * 100:.2f}%")